# Using stable water isotopes for 2-component mass balance hydrograph separation

In [84]:
#################
# LOAD PACKAGES #
#################

library(tidyverse)
library(viridis)
library(dplyr)
library(lubridate)
library(caTools)  # for numerical integration
library(data.table) # for nearest join of q and ISCO data

###################
# SET DIRECTORIES #
###################

setwd("~//OneDrive/git-repos/EMMA/")

input_dir <- "isco_metadata/LCBP_RI_sample_index/"
output_dir <- "isotope-separation/output"
q_input_dir <- "~//OneDrive/git-repos/cQ_analysis/millar2021_R_separation_hysteresis/sonya-may24/data/"

#################
# SET SITE INFO #
#################

site = "Hungerford"
Year = "WY23"

################
# READ IN DATA #
################

# read in the streamwater ISCO data
InputDataISCO <- read.csv(file.path(input_dir, "RI23-isotope-joined.csv"))

# read in the discharge data
InputDataQ <- read.csv(file.path(q_input_dir, "hb_q_chem copy.csv")) %>%
    rename(q_cms = q_cms_hb)
 

########################
# CLEAN AND JOIN DATA  #
########################

# Convert missing values to NA
InputDataISCO[InputDataISCO == ""] <- NA

# Combine 'Date' and 'Time' columns into 'Datetime' 
InputDataISCO$Datetime <- mdy_hm(paste(InputDataISCO$Date, InputDataISCO$Time)) 

# Convert 'datetime' in InputDataQ to proper datetime format
InputDataQ$datetime <- as.POSIXct(InputDataQ$datetime, format="%Y-%m-%dT%H:%M:%SZ")

# Convert dataframes to data.table
InputDataISCO_DT <- as.data.table(InputDataISCO)
InputDataQ_DT <- as.data.table(InputDataQ)

# Set keys for joining
setkey(InputDataISCO_DT, Datetime)
setkey(InputDataQ_DT, datetime)

# Perform a nearest join using data.table
merged_data <- InputDataQ_DT[InputDataISCO_DT, roll = "nearest", on = .(datetime = Datetime)]

###################
# FILTER THE DATA #
###################

InputDataStream <- merged_data %>%
    filter(Site == site) %>%
    filter(!is.na(datetime)) %>%
    filter(Type2 == 'Stream')

InputDataEM <- merged_data %>%
    filter(Site == site) %>%
    filter(!is.na(datetime)) %>%
    filter(Type2 == 'Endmember')

########################
# SEPARATE INTO EVENTS #
########################

# Make sure date formatting all good
InputDataStream$Date <- as.Date(InputDataStream$datetime) # Assuming Datetime is in POSIXct format

# Add Event column based on date ranges
InputDataStream <- InputDataStream %>%
  mutate(Event = case_when(
    Date >= as.Date('2023-02-08') & Date <= as.Date('2023-02-12') ~ 'Event A: 2/09-2/11',
    Date >= as.Date('2023-03-21') & Date <= as.Date('2023-03-25') ~ 'Event B: 3/22-3/25',
    Date >= as.Date('2023-03-30') & Date <= as.Date('2023-04-02') ~ 'Event C: 3/31-4/02',
    Date >= as.Date('2023-04-02') & Date <= as.Date('2023-04-15') ~ 'Event D: 4/02-4/12',
    TRUE ~ NA_character_ # Assign NA to dates outside the defined ranges
  ))

# Filter out rows where Event is NA
InputDataStream <- InputDataStream %>%
  filter(!is.na(Event))

# Nest data by Event
nested_data <- InputDataStream %>%
  group_by(Event) %>%
  nest()

# Create a function to calculate new and old water proportions and plot the hydrograph
process_event <- function(data, event_name, InputDataEM) {
  # Set new and old water values based on the event
  if (event_name == "Event A: 2/09-2/11") {
    Event_new <- InputDataEM %>%
      filter(Type == "Snow lysimeter" & Date == as.Date('2023-02-15')) %>%
      pull(d18O)
    
    Event_old <- InputDataEM %>%
      filter(Type == "Baseflow" & Date == as.Date('2023-01-24')) %>%
      pull(d18O)
  } else if (event_name == "Event B: 3/22-3/25") {
    Event_new <- InputDataEM %>%
      filter(Type == "Snow lysimeter" & Date == as.Date('2022-03-17')) %>%
      pull(d18O)
    
    Event_old <- InputDataEM %>%
      filter(Type == "Baseflow" & Date == as.Date('2023-01-24')) %>%
      pull(d18O)
  } else if (event_name == "Event C: 3/31-4/02") {
    Event_new <- InputDataEM %>%
      filter(Type == "Snow lysimeter" & Date == as.Date('2023-03-28')) %>%
      pull(d18O)
    
    Event_old <- InputDataEM %>%
      filter(Type == "Baseflow" & Date == as.Date('2023-01-24')) %>%
      pull(d18O)
  } else if (event_name == "Event D: 4/02-4/12") {
    Event_new <- InputDataEM %>%
      filter(Type == "Snow lysimeter" & Date == as.Date('2023-04-12')) %>%
      pull(d18O)
    
    Event_old <- InputDataEM %>%
      filter(Type == "Baseflow" & Date == as.Date('2023-01-24')) %>%
      pull(d18O)
  } else {
    stop("Unknown event")
  }

# Calculate Q_o(t) using Equation 3 and ensure it is non-negative
  data <- data %>%
  mutate(Q_o = pmax(`q_cms` * (`d18O` - Event_new) / (Event_old - Event_new), 0),
         Q_n = pmax(`q_cms` - Q_o, 0))
  
  # Plot using ggplot2
  hydrograph_plot <- ggplot(data, aes(x = timestamp)) +
    geom_line(aes(y = q_cms, color = "Total Discharge")) +
    geom_line(aes(y = Q_o, color = "Old Water")) +
    geom_line(aes(y = Q_n, color = "New Water")) +
    scale_color_manual(values = c("Total Discharge" = "blue", "Old Water" = "red", "New Water" = "green")) +
    theme_minimal() +
    labs(title = event_name,
         x = "Datetime",
         y = "Discharge (cms)",
         color = "Components") +
    scale_x_datetime(date_labels = "%Y-%m-%d %H:%M", date_breaks = "1 day") +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))
  
  # Save the plot to a file
  ggsave(file.path(output_dir, paste0("storm_hydrograph_", gsub("[: ]", "_", event_name), ".png")), hydrograph_plot, width = 10, height = 6)
  
  return(hydrograph_plot)
}

# Apply the function to each event
plots <- nested_data %>%
  mutate(plot = map2(data, Event, ~ process_event(.x, .y, InputDataEM)))

# Print all plots 
# Uncomment to print single event plots
#plots$plot

# Arrange the plots in a 2x2 grid
grid.arrange(grobs = plots$plot, ncol = 2)

# Save the 2x2 grid plot
ggsave(file.path(output_dir, paste(Site, Year, "event_IHS_grid.png")), 
       arrangeGrob(grobs = plots$plot, ncol = 2), 
       width = 15, height = 10)

Warning message:
“ 3 failed to parse.”


ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `plot = map2(data, Event, ~process_event(.x, .y, InputDataEM))`.
[1mCaused by error in `map2()`:[22m
[1m[22m[36mℹ[39m In index: 1.
[1mCaused by error in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `Q_o = pmax(q_cms * (d18O - Event_new)/(Event_old - Event_new), 0)`.
[1mCaused by error:[22m
[1m[22m[33m![39m `Q_o` must be size 7 or 1, not 0.


In [82]:
InputDataEM

datetime,q_cms,q_cms_mb,q_cms_pred,q_cms_hb_filled,type,NO3,year,TP,TDP,⋯,Site,Date,Time,Type,Type2,Index.notes,NRS_LWIA_notes,dD,d18O,iso.notes
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<dbl>,<dbl>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
2023-01-04 12:00:00,1.813198,0.9995842,1.615078,1.813198,original,4.522092,2023,0.07018430,0.02394318,⋯,Hungerford,01/04/2023,12:00,Soil water lysimeter wet,Endmember,Wet Soil Lys,NA,-65.565,-9.298,Wet Soil Lys
2023-01-24 12:00:00,1.214586,NA,NA,1.214586,original,3.527281,2023,0.00023500,0.00019880,⋯,Hungerford,01/24/2023,12:00,Baseflow,Endmember,NA,NA,-66.275,-9.682,NA
2023-02-15 12:00:00,2.426009,NA,NA,2.426009,original,4.209841,2023,0.10031978,0.06102721,⋯,Hungerford,02/15/2023,12:00,Soil water lysimeter wet,Endmember,wet site,NA,NA,NA,NA
2023-02-15 12:00:00,2.426009,NA,NA,2.426009,original,4.209841,2023,0.10031978,0.06102721,⋯,Hungerford,02/15/2023,12:00,Snowmelt lysimeter,Endmember,NA,NA,-90.225,-13.668,Wet Site
2023-02-15 12:00:00,2.426009,NA,NA,2.426009,original,4.209841,2023,0.10031978,0.06102721,⋯,Hungerford,02/15/2023,12:00,Soil water lysimeter dry,Endmember,dry site,NA,-79.175,-11.536,Dry lys
2023-02-22 01:15:00,1.821868,1.1836436,1.787191,1.821868,original,3.518227,2023,0.03219521,0.00019880,⋯,Hungerford,02/22/2023,01:15,Grab,Endmember,NA,NA,-68.854,-9.698,NA
2023-03-16 12:00:00,1.214586,0.4615644,1.111978,1.214586,original,2.053002,2023,0.04736297,0.00019880,⋯,Hungerford,03/16/2023,12:00,Soil water lysimeter dry,Endmember,dry site,NA,-81.472,-12.284,Dry lys
2023-03-16 12:00:00,1.214586,0.4615644,1.111978,1.214586,original,2.053002,2023,0.04736297,0.00019880,⋯,Hungerford,03/16/2023,12:00,Groundwater,Endmember,well,NA,-72.633,-10.806,Grab - MED is it gw though?
2023-03-28 12:00:00,2.395498,2.2936634,2.825166,2.395498,original,NA,NA,NA,NA,⋯,Hungerford,03/28/2023,12:00,Snowmelt lysimeter,Endmember,NA,NA,-75.524,-11.193,NA
